## Feasible assignments

This task extracts feasible staff-shift assignments from the SQL database and writes them to a temporary file.

In [ ]:
import pathlib

model_data_path = pathlib.Path("./model_data/").resolve()
db_path = pathlib.Path("./sample_data/staffing.db").resolve()

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine(f"sqlite:///{db_path}")

In [ ]:
query = """
SELECT
    shifts.id as shift_id,
    staff_qualifications.staff_id as staff_id
FROM shifts
INNER JOIN staff_qualifications ON (
    staff_qualifications.qualification_id = shifts.qualification_id
)
INNER JOIN availability ON (
    availability.staff_id == staff_qualifications.staff_id
    AND shifts.start_time >= availability.from_time
    AND shifts.end_time <= availability.to_time
)
INNER JOIN staff on (
    staff_qualifications.staff_id == staff.id
)
WHERE staff.active == 1
"""

with engine.connect() as connection:
    feasible_assignments = pd.read_sql(text(query), connection)

feasible_assignments.to_feather(model_data_path / "feasible_assignments.feather")

In [ ]:
feasible_assignments.sample(4)